<a href="https://colab.research.google.com/github/Sathvik-Ravula/QuantumNN/blob/main/docs/tutorials/mnist_march13-last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!pip install tensorflow==2.15.0

Install TensorFlow Quantum:

In [ ]:
!pip install tensorflow-quantum==0.7.3

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources

importlib.reload(pkg_resources)

Now import TensorFlow and the module dependencies:

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## 1. Load the data

In this tutorial you will build a binary classifier to distinguish between the digits 3 and 6, following <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> This section covers the data handling that:

- Loads the raw data from Keras.
- Filters the dataset to only 3s and 6s.
- Downscales the images so they fit can fit in a quantum computer.
- Removes any contradictory examples.
- Converts the binary images to Cirq circuits.
- Converts the Cirq circuits to TensorFlow Quantum circuits.

### 1 Load the raw data

#HERE IS THE ADAPTATION FOR CRACK DETECTION DATASET

In [ ]:
import tensorflow as tf
import numpy as np
import cirq
import collections
import os

# Define dataset paths
train_dir = "/content/dataset/train"  # Treat 'val' as training data
val_dir = "/content/dataset/val"
test_dir = "/content/dataset/test"

# Load images from directory
def load_images(directory, image_size=(28, 28), batch_size=32):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        image_size=image_size,
        batch_size=batch_size,
        color_mode='grayscale',  # Since original MNIST is grayscale
        shuffle=True
    )
    return dataset

# Load training and testing datasets
train_dataset = load_images(train_dir)
test_dataset = load_images(val_dir)
val_dataset = load_images(test_dir)

# Function to normalize images
def preprocess_image(image, label):
    image = tf.image.resize(image, (4, 4))  # Resize to (4,4)
    image = image / 255.0  # Normalize to [0,1]
    return image, label

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_image)
test_dataset = val_dataset.map(preprocess_image)
val_dataset = test_dataset.map(preprocess_image)

# Convert to numpy arrays
x_train, y_train = [], []
for img, lbl in train_dataset:
    x_train.extend(img.numpy())
    y_train.extend(lbl.numpy())

x_val, y_val = [], []
for img, lbl in train_dataset:
    x_val.extend(img.numpy())
    y_val.extend(lbl.numpy())

x_test, y_test = [], []
for img, lbl in test_dataset:
    x_test.extend(img.numpy())
    y_test.extend(lbl.numpy())

x_train, y_train = np.array(x_train), np.array(y_train)
x_val, y_val = np.array(x_val), np.array(y_val)
x_test, y_test = np.array(x_test), np.array(y_test)

# Convert images to quantum circuits using Angle Encoding
def convert_to_circuit(image):
    """Encodes image using Ry rotations ()."""
    values = np.ndarray.flatten(image)
    qubits = cirq.GridQubit.rect(4, 4)
    circuit = cirq.Circuit()
    for i, value in enumerate(values):
        theta = np.pi * value  # Scale pixel intensity to rotation angle
        circuit.append(cirq.ry(theta)(qubits[i]))
    return circuit

x_train_circ = [convert_to_circuit(x) for x in x_train]
x_val_circ = [convert_to_circuit(x) for x in x_val]
x_test_circ = [convert_to_circuit(x) for x in x_test]


In [ ]:
SVGCircuit(x_train_circ[0])

Compare this circuit to the indices where the image value exceeds the threshold:

In [ ]:
THRESHOLD = 0.5

x_train_bin = np.array(x_train > THRESHOLD, dtype=np.float32)
x_val_bin = np.array(x_val > THRESHOLD, dtype=np.float32)
x_test_bin = np.array(x_test > THRESHOLD, dtype=np.float32)

In [ ]:
bin_img = x_train_bin[0, :, :, 0]
indices = np.array(np.where(bin_img)).T
indices

Convert these `Cirq` circuits to tensors for `tfq`:

In [ ]:
x_train_tfcirc = tfq.convert_to_tensor(x_train_circ)
x_val_tfcirc = tfq.convert_to_tensor(x_val_circ)
x_test_tfcirc = tfq.convert_to_tensor(x_test_circ)

## 2. Quantum neural network

There is little guidance for a quantum circuit structure that classifies images. Since the classification is based on the expectation of the readout qubit, <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> propose using two qubit gates, with the readout qubit always acted upon. This is similar in some ways to running small a <a href="https://arxiv.org/abs/1511.06464" class="external">Unitary RNN</a> across the pixels.

### 2.1 Build the model circuit

This following example shows this layered approach. Each layer uses *n* instances of the same gate, with each of the data qubits acting on the readout qubit.

Start with a simple class that will add a layer of these gates to a circuit:

In [ ]:
class CircuitLayerBuilder():

    def __init__(self, data_qubits, readout):
        self.data_qubits = data_qubits
        self.readout = readout

    def add_layer(self, circuit, gate, prefix):
        for i, qubit in enumerate(self.data_qubits):
            symbol = sympy.Symbol(prefix + '-' + str(i))
            circuit.append(gate(qubit, self.readout)**symbol)

Build an example circuit layer to see how it looks:

In [ ]:
demo_builder = CircuitLayerBuilder(data_qubits=cirq.GridQubit.rect(4, 1),
                                   readout=cirq.GridQubit(-1, -1))

circuit = cirq.Circuit()
demo_builder.add_layer(circuit, gate=cirq.XX, prefix='xx')
SVGCircuit(circuit)

Now build a two-layered model, matching the data-circuit size, and include the preparation and readout operations.

In [ ]:
def create_quantum_model():
    """Create a QNN model circuit and readout operation to go along with it."""
    data_qubits = cirq.GridQubit.rect(4, 4)  # a 4x4 grid.
    readout = cirq.GridQubit(-1, -1)  # a single qubit at [-1,-1]
    circuit = cirq.Circuit()

    # Prepare the readout qubit.
    circuit.append(cirq.X(readout))
    circuit.append(cirq.H(readout))

    builder = CircuitLayerBuilder(data_qubits=data_qubits, readout=readout)

    # Then add layers (experiment by adding more).
    builder.add_layer(circuit, cirq.XX, "xx1")
    builder.add_layer(circuit, cirq.ZZ, "zz1")

    # Finally, prepare the readout qubit.
    circuit.append(cirq.H(readout))

    return circuit, cirq.Z(readout)

In [ ]:
import cirq
import sympy

def create_better_quantum_model():
    """Create an improved QNN with more expressive layers and deeper entanglement."""
    data_qubits = cirq.GridQubit.rect(4, 4)  # 4x4 grid
    readout = cirq.GridQubit(-1, -1)  # Readout qubit
    circuit = cirq.Circuit()

    # Prepare the readout qubit
    circuit.append([cirq.X(readout), cirq.H(readout)])

    # Introduce variational parameters
    symbols = [sympy.Symbol(f'theta_{i}') for i in range(len(data_qubits) * 2)]

    # Apply parameterized single-qubit rotations
    for i, qubit in enumerate(data_qubits):
        circuit.append(cirq.ry(symbols[i])(qubit))
        circuit.append(cirq.rz(symbols[i + len(data_qubits)])(qubit))

    # Apply multiple layers of entanglement (improves non-linearity)
    for _ in range(3):  # Increase depth by repeating layers
        circuit.append(cirq.XX(q1, q2) for q1, q2 in zip(data_qubits[:-1], data_qubits[1:]))
        circuit.append(cirq.ZZ(q1, q2) for q1, q2 in zip(data_qubits[:-1], data_qubits[1:]))

    # Final readout qubit preparation
    circuit.append(cirq.H(readout))

    return circuit, cirq.Z(readout)


In [ ]:
model_circuit, model_readout = create_quantum_model()

In [ ]:
model_circuit

### 2.2 Wrap the model-circuit in a tfq-keras model

Build the Keras model with the quantum components. This model is fed the "quantum data", from `x_train_circ`, that encodes the classical data. It uses a *Parametrized Quantum Circuit* layer, `tfq.layers.PQC`, to train the model circuit, on the quantum data.

To classify these images, <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> proposed taking the expectation of a readout qubit in a parameterized circuit. The expectation returns a value between 1 and -1.

In [ ]:
# Build the Keras model.
model = tf.keras.Sequential([
    # The input is the data-circuit, encoded as a tf.string
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    # The PQC layer returns the expected value of the readout gate, range [-1,1].
    tfq.layers.PQC(model_circuit, model_readout),
])

Next, describe the training procedure to the model, using the `compile` method.

Since the the expected readout is in the range `[-1,1]`, optimizing the hinge loss is a somewhat natural fit.

Note: Another valid approach would be to shift the output range to `[0,1]`, and treat it as the probability the model assigns to class `3`. This could be used with a standard a `tf.losses.BinaryCrossentropy` loss.

To use the hinge loss here you need to make two small adjustments. First convert the labels, `y_train_nocon`, from boolean to `[-1,1]`, as expected by the hinge loss.

In [ ]:
y_train_hinge = 2.0 * y_train - 1.0
y_val_hinge = 2.0 * y_val - 1.0
y_test_hinge = 2.0 * y_test - 1.0

Second, use a custiom `hinge_accuracy` metric that correctly handles `[-1, 1]` as the `y_true` labels argument.
`tf.losses.BinaryAccuracy(threshold=0.0)` expects `y_true` to be a boolean, and so can't be used with hinge loss).

In [ ]:
def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)

In [ ]:
model.compile(loss=tf.keras.losses.Hinge(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[hinge_accuracy])

In [ ]:
model.load_weights("/content/quantum_model_weights-testacc98.h5")

In [ ]:
print(model.summary())

### Train the quantum model

Now train the model—this takes about 45 min. If you don't want to wait that long, use a small subset of the data (set `NUM_EXAMPLES=500`, below). This doesn't really affect the model's progress during training (it only has 32 parameters, and doesn't need much data to constrain these). Using fewer examples just ends training earlier (5min), but runs long enough to show that it is making progress in the validation logs.

In [ ]:
EPOCHS = 25
BATCH_SIZE = 32

NUM_EXAMPLES = len(x_train_tfcirc)

In [ ]:
x_train_tfcirc_sub = x_train_tfcirc[:NUM_EXAMPLES]
y_train_hinge_sub = y_train_hinge[:NUM_EXAMPLES]

Training this model to convergence should achieve >85% accuracy on the test set.

In [ ]:
qnn_history = model.fit(x_train_tfcirc_sub,
                        y_train_hinge_sub,
                        batch_size=32,
                        epochs=EPOCHS,
                        verbose=1,
                        validation_data=(x_val_tfcirc, y_val_hinge))
print("Evaluating on Test")
qnn_results = model.evaluate(x_test_tfcirc, y_test)

In [ ]:
qnn_results = model.evaluate(x_test_tfcirc, y_test)

Note: The training accuracy reports the average over the epoch. The validation accuracy is evaluated at the end of each epoch.

In [ ]:
# Save model weights
#model.save_weights("quantum_model_weights-20epochstestacc98.h5")


#USE MODEL TO PREDICT

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Image as IPImage
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import numpy as np
from PIL import Image
import io

def upload_and_classify(model):
    """Upload an image file and classify it using the quantum model."""
    uploader = widgets.FileUpload(accept='image/*', multiple=False)
    display(uploader)

    def on_upload(change):
        """Process uploaded image."""
        uploaded_file = next(iter(uploader.value.values()))
        image_data = uploaded_file['content']

        image_path = "/content/uploaded_image.jpg"
        with open(image_path, "wb") as f:
            f.write(image_data)

        print("\nUploaded Image:")
        display(IPImage(image_path))

        result, score = classify_image(model, image_path)
        print("\n🔍 Final Classification Result:", result)
        print("📊 Model Confidence Score:", score)

    uploader.observe(on_upload, names='value')

def load_and_preprocess_image(image_path, image_size=(4, 4)):
    """Loads an image, converts to grayscale, resizes, and normalizes it."""
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = image.resize(image_size)  # Resize to 4x4
    image_array = np.array(image) / 255.0
    return image_array


def classify_image(model, image_path):
    """Takes an image file, processes it, and returns a classification result."""
    image = load_and_preprocess_image(image_path)
    circuit = convert_to_circuit(image)
    x_input = convert_circuit_to_tensor(circuit)

    prediction = model(x_input)
    score = prediction.numpy()[0][0]


    class_names = tf.keras.utils.image_dataset_from_directory("/content/dataset/train").class_names
    print("Class Names:", class_names)

    if class_names[0] == "crack":
        crack_label = -1
        good_label = +1
    else:
        crack_label = +1
        good_label = -1

    scaled_score = 2 * score  # Expand range to [-2, 2]
    if scaled_score > 1:
        result = "Good (No Crack)"
    else:
        result = "Crack Detected"

    # Print Model Output
    print()
    print(f"Score: {score} | Classification: {result}")

    # Visualize Quantum Circuit
    #print("\nQuantum Circuit Representation:")
    #print(circuit)

    return result, score


upload_and_classify(model)


## 3. Classical neural network

While the quantum neural network works for this simplified MNIST problem, a basic classical neural network can easily outperform a QNN on this task. After a single epoch, a classical neural network can achieve >98% accuracy on the holdout set.

In the following example, a classical neural network is used for for the 3-6 classification problem using the entire 28x28 image instead of subsampling the image. This easily converges to nearly 100% accuracy of the test set.

In [ ]:
def create_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.Conv2D(32, [3, 3],
                               activation='relu',
                               input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1))
    return model


model = create_classical_model()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Reload training and testing datasets in original (28,28,1) size
train_dataset_cnn = load_images("/content/dataset/train", image_size=(28, 28))  # Use original size
val_dataset_cnn = load_images("/content/dataset/val", image_size=(28, 28))
test_dataset_cnn = load_images("/content/dataset/test", image_size=(28,28))

# Convert to numpy arrays
x_train_cnn, y_train_cnn = [], []
for img, lbl in train_dataset_cnn:
    x_train_cnn.extend(img.numpy())
    y_train_cnn.extend(lbl.numpy())

x_val_cnn, y_val_cnn = [], []
for img, lbl in val_dataset_cnn:
    x_val_cnn.extend(img.numpy())
    y_val_cnn.extend(lbl.numpy())

x_test_cnn, y_test_cnn = [], []
for img, lbl in test_dataset_cnn:
    x_test_cnn.extend(img.numpy())
    y_test_cnn.extend(lbl.numpy())

x_train_cnn, y_train_cnn = np.array(x_train_cnn), np.array(y_train_cnn)
x_val_cnn, y_val_cnn = np.array(x_val_cnn), np.array(y_val_cnn)
x_test_cnn, y_test_cnn = np.array(x_test_cnn), np.array(y_test_cnn)

# training the CNN
model.fit(x_train_cnn, y_train_cnn, batch_size=128, epochs=10, verbose=1, validation_data=(x_val_cnn, y_val_cnn))

print("Evaluating on Test")
cnn_results = model.evaluate(x_test_cnn, y_test_cnn)

The above model has nearly 1.2M parameters. For a more fair comparison, try a 37-parameter model, on the subsampled images:

In [ ]:
def create_fair_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(4, 4, 1)))
    model.add(tf.keras.layers.Dense(2, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    return model


model = create_fair_classical_model()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(x_train_bin,
          y_train,
          batch_size=128,
          epochs=10,
          verbose=2,
          validation_data=(x_val_bin, y_val))
print("Evaluating on Test")
fair_nn_results = model.evaluate(x_test_bin, y_test)

## 4. Comparison

Higher resolution input and a more powerful model make this problem easy for the CNN. While a classical model of similar power (~32 parameters) trains to a similar accuracy in a fraction of the time. One way or the other, the classical neural network easily outperforms the quantum neural network. For classical data, it is difficult to beat a classical neural network.

In [ ]:
qnn_accuracy = qnn_results[1]
cnn_accuracy = cnn_results[1]
fair_nn_accuracy = fair_nn_results[1]

sns.barplot(x=["Quantum", "Classical, full", "Classical, fair"],
            y=[qnn_accuracy, cnn_accuracy, fair_nn_accuracy])